In [1]:
import findspark
findspark.init()

from pyspark.sql import functions as F

In [2]:
from pyspark.sql import SparkSession
from itertools import chain

import pandas as pd
pd.options.display.max_rows = 100

# Configure Spark session

In [3]:
def run():
    from pyspark import SparkContext, SparkConf

    conf = SparkConf()
    conf.setAppName('cars')
    conf.set("spark.driver.memory", "1g")
    conf.set("spark.driver.memoryOverhead", "0.10")
    sc = SparkContext(conf=conf)
    
    return sc

spark = SparkSession(run())

In [4]:
# turn on a dataframe view
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [ ]:
# Summary function
def user_activity_workout_summarize(df):
    user_count = format(df.select('userId').distinct().count(), ',d')
    workout_count = format(df.select('id').distinct().count(), ',d')
    activity_count = str(df.select('sport').distinct().count())
    seqOp = (lambda x,y: x+y)
    sum_temp = df.rdd.map(lambda x: len(x.timestamp)).aggregate(0, seqOp, seqOp)
    total_records_count = format(sum_temp, ',d')
    columns=['Users count', 'Activity types count','Workouts count', 'Total records count']
    data = [[user_count], [activity_count], [workout_count], [total_records_count]]
    sum_dict = {column: data[i] for i, column in enumerate(columns)}
    sum_df = pd.DataFrame.from_dict(sum_dict)[columns]
    gender_user_count = df.select('gender','userId').distinct().groupBy('gender').count().toPandas()
    gender_activities_count = df.groupBy('gender').count().toPandas()
    gender_user_activity_count = gender_user_count.join(
        gender_activities_count.set_index('gender'), on='gender'
        , how='inner', lsuffix='_gu'
    )
    gender_user_activity_count.columns = ['Gender', '# of users', 'Activities (workouts) count']
    
    return sum_df, gender_user_activity_count

sum_dfs = user_activity_workout_summarize(df)
print('\nOverall data set summary on users, activities(workouts) and number of fitness records:')
sum_dfs[0]
print('\nSummarize on genders:')
sum_dfs[1]

# Load data

In [5]:
df = spark.read.options(delimiter = '.').csv(path = "used_cars_data.csv", sep = ',',  header = True)

In [6]:
display(df)

vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,daysonmarket,dealer_zip,description,engine_cylinders,engine_displacement,engine_type,exterior_color,fleet,frame_damaged,franchise_dealer,franchise_make,front_legroom,fuel_tank_volume,fuel_type,has_accidents,height,highway_fuel_economy,horsepower,interior_color,isCab,is_certified,is_cpo,is_new,is_oemcpo,latitude,length,listed_date,listing_color,listing_id,longitude,main_picture_url,major_options,make_name,maximum_seating,mileage,model_name,owner_count,power,price,salvage,savings_amount,seller_rating,sp_id,sp_name,theft_title,torque,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
ZACNJABB5KPJ92081,35.1 in,null,null,null,SUV / Crossover,null,Bayamon,null,null,522,00960,[!@@Additional In...,I4,1300.0,I4,Solar Yellow,null,null,True,Jeep,41.2 in,12.7 gal,Gasoline,null,66.5 in,null,177.0,Black,null,null,null,True,null,18.3988,166.6 in,2019-04-06,YELLOW,237132766,-66.1582,https://static.ca...,['Quick Order Pac...,Jeep,5 seats,7.0,Renegade,null,"177 hp @ 5,750 RPM",23141.0,null,0,2.8,370599,Flagship Chrysler,null,"200 lb-ft @ 1,750...",A,9-Speed Automatic...,t83804,Latitude FWD,null,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
SALCJ2FX1LH858117,38.1 in,null,null,null,SUV / Crossover,null,San Juan,null,null,207,00922,[!@@Additional In...,I4,2000.0,I4,Narvik Black,null,null,True,Land Rover,39.1 in,17.7 gal,Gasoline,null,68 in,null,246.0,Black (Ebony),null,null,null,True,null,18.4439,181 in,2020-02-15,BLACK,265946296,-66.0785,https://static.ca...,['Adaptive Cruise...,Land Rover,7 seats,8.0,Discovery Sport,null,"246 hp @ 5,500 RPM",46500.0,null,0,3.0,389227,Land Rover San Juan,null,"269 lb-ft @ 1,400...",A,9-Speed Automatic...,t86759,S AWD,null,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
JF1VA2M67G9829723,35.4 in,null,null,null,Sedan,null,Guaynabo,17.0,null,1233,00969,null,H4,2500.0,H4,None,False,False,True,FIAT,43.3 in,15.9 gal,Gasoline,False,58.1 in,23.0,305.0,None,False,null,null,False,null,18.3467,180.9 in,2017-04-25,UNKNOWN,173473508,-66.1098,null,"['Alloy Wheels', ...",Subaru,5 seats,null,WRX STI,3.0,"305 hp @ 6,000 RPM",46995.0,False,0,null,370467,FIAT de San Juan,False,"290 lb-ft @ 4,000...",M,6-Speed Manual,t58994,Base,null,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
SALRR2RV0L2433391,37.6 in,null,null,null,SUV / Crossover,null,San Juan,null,null,196,00922,[!@@Additional In...,V6,3000.0,V6,Eiger Gray,null,null,True,Land Rover,39 in,23.5 gal,Gasoline,null,73 in,null,340.0,Gray (Ebony/Ebony...,null,null,null,True,null,18.4439,195.1 in,2020-02-26,GRAY,266911050,-66.0785,https://static.ca...,null,Land Rover,7 seats,11.0,Discovery,null,"340 hp @ 6,500 RPM",67430.0,null,0,3.0,389227,Land Rover San Juan,null,"332 lb-ft @ 3,500...",A,8-Speed Automatic...,t86074,V6 HSE AWD,null,AWD,All-Wheel Drive,115 in,87.4 in,2020
SALCJ2FXXLH862327,38.1 in,null,null,null,SUV / Crossover,null,San Juan,null,null,137,00922,[!@@Additional In...,I4,2000.0,I4,Narvik Black,null,null,True,Land Rover,39.1 in,17.7 gal,Gasoline,null,68 in,null,246.0,Black (Ebony),null,null,null,True,null,18.4439,181 in,2020-04-25,BLACK,270957414,-66.0785,https://static.ca...,['Adaptive Cruise...,Land Rover,7 seats,7.0,Discovery Sport,null,"246 hp @ 5,500 RPM",48880.0,null,0,3.0,389227,Land Rover San Juan,null,"269 lb-ft @ 1,400...",A,9-Speed Automatic...,t86759,S AWD,null,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
SALYK2EX1LA261711,37.1 in,null,null,null,SUV / Crossover,null,San Juan,null,null,242,00922,[!@@Additional In...,I4,2000.0,I4,Kaikoura Stone,False,False,True,Land Rover,40.2 in,16.6 gal,Gasoline,False,66.3 in,null,247.0,Brown (Ebony / Eb...,False,null,null,True,null,18.4439,188.9 in,2020-01-11,UNKNOWN,262940541,-66.0785,https://static.ca...,"['Leather Seats',...",Land Rover,5 seats,12.0,Range Rover Velar,null,"247 hp @ 5,500 RPM",66903.0,False,0,3.0,389227,Land Rover San Juan,False,"269 lb-ft @ 1

In [7]:
# save to parquet
df.write.parquet("/tmp/parquet/cars.parquet")

In [8]:
# read from parquet
parDF=spark.read.parquet("/tmp/parquet/cars.parquet")

# Data overview

In [9]:
# shape
print(format(parDF.count(), ',d'), '\ncol cnt:', len(parDF.columns))

3,000,507 
col cnt: 66


In [10]:
# data types
print('Columns & datatypes:')
pd.DataFrame(parDF.dtypes, columns =['col_name', 'data_type'])

Columns & datatypes:


,col_name,data_type
0,vin,string
1,back_legroom,string
2,bed,string
3,bed_height,string
4,bed_length,string
5,body_type,string
6,cabin,string
7,city,string
8,city_fuel_economy,string
9,combine_fuel_economy,string


###### get count from descriptive stats

In [11]:
# descriptive 
descr = parDF.describe().toPandas()
descr

,summary,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,count,3000507,2840909,19679,429196,429191,2986589,63622,3000133,2508829,...,2918550,2912147,2880702,2884802,1158444,2853203,2848662,2828164,2830498,2946381
1,mean,Infinity,None,4081.25,5166.666666666667,63740.0,5000.0,36161.2,23104.127042857144,22.706573085279945,...,6938472.700991163,9321036.134121323,1.1656370838791057E7,1.1037576677955545E7,1.0958303484245954E7,1.267889072667737E7,1.3396662595330501E7,1.4679855323887112E7,1.3845013514002014E7,342090.8502279081
2,stddev,NaN,None,1653.3677751788923,1474.2229591663988,109272.00131781244,0.0,47620.57877850709,53347.21956367153,16.079865342342412,...,4.298426253523661E7,4.970049252064884E7,5.538545201250243E7,5.397217480572898E7,5.374599443231616E7,5.7648088615293436E7,5.918315832965848E7,6.178457258018881E7,6.009541186023207E7,9667363.89268572
3,min,Heated Windshield Washer nozzles,WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,'Blind Spot Monitoring',WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,& belt. Recently changed the oil,WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,...,,,,,,,,,,
4,max,type: SUV,V8 Flex Fuel Vehicle,Volkswagen,V8 Flex Fuel Vehicle,White,White,V8,private seller,V8,...,~BACK UP CAMERA~,~~ VERY LOW MILES ~~,~GM CERTIFIED,~~~ Call (502) 695-9050 to check availability ...,~~~ Call (502) 695-9050 to check availability ...,~PUSH BUTTON START~,~~~ Call (502) 695-9050 to check availability ...,~PUSH BUTTON START~,~GM CERTIFIED,~~~ Call (502) 695-9050 to check availability ...


In [12]:
# get count (descending order)
descr_trans = descr.T
descr_trans.columns = descr_trans.iloc[0]

descr_trans = descr_trans.drop(descr_trans.index[0])
descr_trans['count'] = descr_trans['count'].astype('int64') 
descr_trans['count'].sort_values(ascending=False)

vin                        3000507
city                       3000133
daysonmarket               3000126
dealer_zip                 3000126
exterior_color             2996558
body_type                  2986589
franchise_dealer           2985724
model_name                 2975734
interior_color             2973804
make_name                  2972421
price                      2969827
sp_name                    2967403
longitude                  2964886
sp_id                      2963782
savings_amount             2959047
listing_id                 2954554
year                       2946381
listing_color              2939284
is_new                     2935768
seller_rating              2931478
mileage                    2929166
engine_cylinders           2928909
engine_type                2926986
listed_date                2925745
description                2922226
fuel_type                  2921440
transmission               2918550
transmission_display       2912147
latitude            

In [13]:
parDF.where(F.col('city').isNotNull()).count()

3000133

###### get top n values by count from every column

In [14]:
descr_trans

summary,count,mean,stddev,min,max
vin,3000507,Infinity,NaN,Heated Windshield Washer nozzles,type: SUV
back_legroom,2840909,None,None,WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,V8 Flex Fuel Vehicle
bed,19679,4081.25,1653.3677751788923,WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,Volkswagen
bed_height,429196,5166.666666666667,1474.2229591663988,WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,V8 Flex Fuel Vehicle
bed_length,429191,63740.0,109272.00131781244,WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,White
body_type,2986589,5000.0,0.0,'Blind Spot Monitoring',White
cabin,63622,36161.2,47620.57877850709,WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,V8
city,3000133,23104.127042857144,53347.21956367153,& belt. Recently changed the oil,private seller
city_fuel_economy,2508829,22.706573085279945,16.079865342342412,WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,V8
combine_fuel_economy,86,6034.0,6862.30777508558,'Backup Camera',V6


In [15]:
n=10

def get_value_cnt(col, n=n):

    cnt = dict(parDF.select(col).groupBy(col).count().orderBy('count', ascending=False).take(n))
    return cnt


descr_trans['top_n_values'] = [get_value_cnt(col) for col in parDF.columns]

In [16]:
pd.set_option('display.max_colwidth', None)
descr_trans[['count', 'top_n_values']].sort_values(by='count', ascending=False)

summary,count,top_n_values
vin,3000507,"{' Sales Department open until 7 PM on Mondays. Doing business with Fairmont Ford will put a smile on your face. Established in 1909': 15, '5FNYF6H94MB000906': 2, '2HKRW2H52LH679102': 2, '2FMPK4K97LBB07668': 2, '5FNYF6H10MB018426': 2, '1FADP3F29FL349680': 2, '5N1AZ2CSXLN157394': 2, '5XYPG4A31KG501728': 2, '1N4BL4DV0LC262865': 2, '1N6AD0EV6KN713209': 2}"
city,3000133,"{'Houston': 43777, 'San Antonio': 24947, 'Columbus': 18333, 'Miami': 18198, 'Jacksonville': 16837, 'Las Vegas': 16396, 'Tampa': 15353, 'Phoenix': 14791, 'Dallas': 14641, 'Orlando': 14499}"
daysonmarket,3000126,"{'8': 73158, '7': 72604, '6': 69285, '13': 58356, '14': 58340, '5': 57396, '15': 56730, '12': 53006, '0': 52296, '20': 50989}"
dealer_zip,3000126,"{'77477': 7320, '33619': 5619, '77034': 5257, '77074': 5217, '91401': 5113, '77090': 4942, '85382': 4801, '85297': 4739, '85260': 4717, '30096': 4621}"
exterior_color,2996558,"{'Black': 105464, 'White': 95246, 'Silver': 59396, 'Gray': 58950, 'None': 40261, 'Summit White': 40244, 'Blue': 39021, 'Red': 31466, 'I4': 31330, '4-Wheel Disc Brakes': 30845}"
body_type,2986589,"{'SUV / Crossover': 1416402, 'Sedan': 742036, 'Pickup Truck': 474595, 'Hatchback': 88374, 'Minivan': 79802, 'Coupe': 71607, 'Van': 47166, 'Wagon': 40505, 'Convertible': 26010, None: 13918}"
franchise_dealer,2985724,"{'True': 1327898, 'False': 511280, 'Electronic Stability Control': 26082, 'Air Conditioning': 23666, 'I4': 21412, '4-Wheel Disc Brakes': 19916, None: 14783, 'Front Bucket Seats': 13819, 'V6': 13184, '6 Speakers': 12722}"
model_name,2975734,"{'F-150': 64858, 'Escape': 38920, 'Camry': 33630, 'Rogue': 31678, 'Equinox': 30903, 'Silverado 1500': 30495, 'CR-V': 30245, 'Accord': 29354, 'Civic': 27664, 'Explorer': 27629}"
interior_color,2973804,"{'Black': 498405, 'None': 315687, 'Gray': 153803, 'Black (Ebony)': 75200, 'Jet Black': 69520, 'Black (Charcoal)': 65757, 'Brown (Beige)': 31947, None: 26703, 'Brown (Tan)': 25656, 'False': 24594}"
make_name,2972421,"{'Ford': 286435, 'Chevrolet': 188171, 'Toyota': 169544, 'Honda': 158464, 'Nissan': 135681, 'Hyundai': 103593, 'Jeep': 77151, 'Kia': 75628, 'GMC': 50137, 'Volkswagen': 46596}"


In [16]:
# parDF.select('bed').groupBy('bed').count().orderBy('count', ascending=False).select('bed').take(20)

###### Columns below won't be investigated:

vin,
longitude,
savings_amount,
listing_id,
description,
latitude,
trimId,
main_picture_url,
frame_damaged,
theft_title,
salvage,
is_certified,
vehicle_damage_category,
bed_height,
bed_length,
cabin,
bed,
combine_fuel_economy,


# Data extraction

#### boolean variables

if True in string = 1

if False in string = 0

something else = -1

In [17]:
# list of boolean variables
bool_v = ['franchise_dealer', 'is_new', 'fleet', 'has_accidents', 'isCab', 'is_cpo', 'is_oemcpo']

In [18]:
# extract 1, 0, -1
for col in bool_v:

    parDF = parDF.withColumn('is_{}'.format(col),\
         F.when(\
                F.lower(\
                F.col('{}'.format(col))).contains('true'), F.lit(1))\
                             .when(\
                                    F.lower(\
                                    F.col('{}'.format(col))).contains('false'), F.lit(0)).otherwise(F.lit(-1)))

In [50]:
# check

def check(prefix, variables):
    
    is_cols = [prefix + x for x in variables]
    source_vs_is = list(chain.from_iterable(zip(variables, is_cols)))

    return parDF.select(source_vs_is).limit(20).toPandas()

check('is_', bool_v)

,franchise_dealer,is_franchise_dealer,is_new,is_is_new,fleet,is_fleet,has_accidents,is_has_accidents,isCab,is_isCab,is_cpo,is_is_cpo,is_oemcpo,is_is_oemcpo
0,True,1,True,1,None,-1,None,-1,None,-1,None,-1,None,-1
1,True,1,True,1,False,0,False,0,False,0,None,-1,None,-1
2,True,1,True,1,None,-1,None,-1,None,-1,None,-1,None,-1
3,True,1,True,1,None,-1,None,-1,None,-1,None,-1,None,-1
4,True,1,False,0,True,1,False,0,True,1,None,-1,None,-1
5,True,1,False,0,False,0,False,0,False,0,None,-1,None,-1
6,True,1,True,1,False,0,False,0,False,0,None,-1,None,-1
7,True,1,True,1,None,-1,None,-1,None,-1,None,-1,None,-1
8,False,0,False,0,True,1,False,0,True,1,None,-1,None,-1
9,True,1,False,0,False,0,False,0,False,0,None,-1,None,-1


#### continous varaibles

In [45]:
# # powierzchnia całkowita
# def parse_value(val, unit="in"):
    
#     try:
#         if isinstance(val, int): return val
#         if isinstance(val, float): return val

#         if unit:
#             if '.' in val:
#                 return float(val.split(unit)[0].replace(" ", ""))
#             return int(val.split(unit)[0].replace(" ", ""))

#         if '.' in val:
#             return float(val)
#         return int(val)
    
#     except:
#         return None

In [93]:
parDF.withColumn("ext_daysonmarket", F.regexp_extract('daysonmarket', '\d+' , 0)).select('ext_daysonmarket', 'daysonmarket').show()

+----------------+------------+
|ext_daysonmarket|daysonmarket|
+----------------+------------+
|             314|         314|
|              57|          57|
|              72|          72|
|              75|          75|
|              83|          83|
|              33|          33|
|             448|         448|
|              13|          13|
|               6|           6|
|              39|          39|
|             227|         227|
|              82|          82|
|               2|           2|
|              89|          89|
|              47|          47|
|              75|          75|
|               7|           7|
|             356|         356|
|              12|          12|
|              36|          36|
+----------------+------------+
only showing top 20 rows



In [47]:
conti_var = ['daysonmarket'
             ,'price'
             ,'year'
             ,'mileage'
             ,'engine_displacement'
             ,'height'
             ,'fuel_tank_volume'
             ,'front_legroom'
             ,'horsepower'
             ,'back_legroom'
             ,'width'
             ,'wheelbase'
             ,'length'
             ,'city_fuel_economy']

In [53]:
for col in conti_var:
    parDF = parDF.withColumn("conti_{}".format(col), F.regexp_extract('{}'.format(col), '\d+(?:.\d+)?' , 0))

In [54]:
check('conti_', conti_var)

,daysonmarket,conti_daysonmarket,price,conti_price,year,conti_year,mileage,conti_mileage,engine_displacement,conti_engine_displacement,...,back_legroom,conti_back_legroom,width,conti_width,wheelbase,conti_wheelbase,length,conti_length,city_fuel_economy,conti_city_fuel_economy
0,314,314,23006.0,23006.0,2020,2020,5.0,5.0,2000.0,2000.0,...,38.2 in,38.2,72.8 in,72.8,105.1 in,105.1,176.4 in,176.4,22.0,22.0
1,57,57,25911.0,25911.0,2020,2020,8.0,8.0,2500.0,2500.0,...,38.3 in,38.3,83.5 in,83.5,112.2 in,112.2,191.7 in,191.7,23.0,23.0
2,72,72,61305.0,61305.0,2020,2020,3.0,3.0,3600.0,3600.0,...,38.9 in,38.9,89.3 in,89.3,120.9 in,120.9,204.3 in,204.3,17.0,17.0
3,75,75,27565.0,27565.0,2020,2020,1101.0,1101.0,1200.0,1200.0,...,36 in,36,71.4 in,71.4,102.2 in,102.2,171.4 in,171.4,26.0,26.0
4,83,83,18995.0,18995.0,2019,2019,13994.0,13994.0,2000.0,2000.0,...,36.7 in,36.7,81 in,81,99.2 in,99.2,161.3 in,161.3,23.0,23.0
5,33,33,58994.0,58994.0,2018,2018,46774.0,46774.0,6200.0,6200.0,...,39 in,39,80.5 in,80.5,116 in,116,203.9 in,203.9,14.0,14.0
6,448,448,25942.0,25942.0,2019,2019,8.0,8.0,2000.0,2000.0,...,36.7 in,36.7,81 in,81,99.2 in,99.2,161.3 in,161.3,23.0,23.0
7,13,13,26860.0,26860.0,2021,2021,5.0,5.0,None,None,...,38.2 in,38.2,72.8 in,72.8,105.1 in,105.1,176.4 in,176.4,None,None
8,6,6,13942.0,13942.0,2014,2014,73145.0,73145.0,3500.0,3500.0,...,34.6 in,34.6,73.2 in,73.2,109.3 in,109.3,190.6 in,190.6,19.0,19.0
9,39,39,21993.0,21993.0,2020,2020,10226.0,10226.0,2000.0,2000.0,...,38.2 in,38.2,72.8 in,72.8,105.1 in,105.1,176.4 in,176.4,22.0,22.0


In [38]:
import re
re.findall("\d+\.\d+", "12.0")

['12.0']

In [46]:
re.findall("\d+(?:.\d+)?", "12 ljnjn")

['12']

In [37]:
re.findall("[+-]?[0-9]+\.[0-9]+", "12.67 89.0lk")

['12.67', '89.0']

In [81]:
# Function to extract all the numbers from the given string
def getNumbers(str):
    array = re.findall(r'[0-9]+', str)
    return array

getNumbers('6876jnjnj554')

['6876', '554']

In [ ]:
https://stackoverflow.com/questions/32571348/getting-only-the-first-number-from-string-in-python